In [26]:
import torch

In [27]:
with open('../data/tinyshakespeare/input.txt','r',encoding='utf-8') as f:
    text = f.read()

In [28]:
print('Total rows in text',len(text))

Total rows in text 1115394


In [29]:
text[:1000]

"First Citizen:\nBefore we proceed any further, hear me speak.\n\nAll:\nSpeak, speak.\n\nFirst Citizen:\nYou are all resolved rather to die than to famish?\n\nAll:\nResolved. resolved.\n\nFirst Citizen:\nFirst, you know Caius Marcius is chief enemy to the people.\n\nAll:\nWe know't, we know't.\n\nFirst Citizen:\nLet us kill him, and we'll have corn at our own price.\nIs't a verdict?\n\nAll:\nNo more talking on't; let it be done: away, away!\n\nSecond Citizen:\nOne word, good citizens.\n\nFirst Citizen:\nWe are accounted poor citizens, the patricians good.\nWhat authority surfeits on would relieve us: if they\nwould yield us but the superfluity, while it were\nwholesome, we might guess they relieved us humanely;\nbut they think we are too dear: the leanness that\nafflicts us, the object of our misery, is as an\ninventory to particularise their abundance; our\nsufferance is a gain to them Let us revenge this with\nour pikes, ere we become rakes: for the gods know I\nspeak this in hunger 

In [30]:
# build vocabulary table
chars = sorted(list(set(text)))
vocab_size = len(chars)
print(f'chars are: {''.join(chars)}')
print(f'vocab_size is: {vocab_size}')

chars are: 
 !$&',-.3:;?ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz
vocab_size is: 65


In [31]:
# build tokenizer
itos ={i:ch for i,ch in enumerate(chars)}
stoi ={ch:i for i,ch in enumerate(chars)}

encode = lambda s:[stoi[c] for c in s]
decode = lambda l:''.join([itos[i] for i in l])

In [32]:
print('hi there:',decode(encode('hi there')))

hi there: hi there


In [33]:
data = torch.tensor(encode(text),dtype=torch.long)
print(f'data shape and dtype are: {data.shape} {data.dtype}')
print(data[:10])

data shape and dtype are: torch.Size([1115394]) torch.int64
tensor([18, 47, 56, 57, 58,  1, 15, 47, 58, 47])


In [34]:
# 将数据分割成训练集和测试集
n = int(len(data) * 0.9)
train_data = data[:n]
val_data = data[n:]

In [35]:
# 上下文长度
block_size = 8
# 9个训练数据实际包含了8组训练数据
train_data[:block_size+1]

tensor([18, 47, 56, 57, 58,  1, 15, 47, 58])

In [36]:
x = train_data[:block_size]
y = train_data[1:block_size+1]
for t in range(block_size):
    context = x[:t+1]
    target = y[t]
    print(f'when input sequence is {context}, the target is {target}')

when input sequence is tensor([18]), the target is 47
when input sequence is tensor([18, 47]), the target is 56
when input sequence is tensor([18, 47, 56]), the target is 57
when input sequence is tensor([18, 47, 56, 57]), the target is 58
when input sequence is tensor([18, 47, 56, 57, 58]), the target is 1
when input sequence is tensor([18, 47, 56, 57, 58,  1]), the target is 15
when input sequence is tensor([18, 47, 56, 57, 58,  1, 15]), the target is 47
when input sequence is tensor([18, 47, 56, 57, 58,  1, 15, 47]), the target is 58


In [37]:
# build batch，通过batch可以加速训练过程
torch.manual_seed(1337)
batch_size = 4
block_size = 8

def get_batch(split):
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data)-block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    return x, y

In [38]:
xb,yb = get_batch('train')
print(f'x is:{xb}')
print(f'y is:{yb}')

print('-'*30)
for b in range(batch_size):
    for t in range(block_size):
        context = xb[b][:t+1]
        target = yb[b][t]
        print(f'when input sequence is {context}, the target is {target}')

x is:tensor([[24, 43, 58,  5, 57,  1, 46, 43],
        [44, 53, 56,  1, 58, 46, 39, 58],
        [52, 58,  1, 58, 46, 39, 58,  1],
        [25, 17, 27, 10,  0, 21,  1, 54]])
y is:tensor([[43, 58,  5, 57,  1, 46, 43, 39],
        [53, 56,  1, 58, 46, 39, 58,  1],
        [58,  1, 58, 46, 39, 58,  1, 46],
        [17, 27, 10,  0, 21,  1, 54, 39]])
------------------------------
when input sequence is tensor([24]), the target is 43
when input sequence is tensor([24, 43]), the target is 58
when input sequence is tensor([24, 43, 58]), the target is 5
when input sequence is tensor([24, 43, 58,  5]), the target is 57
when input sequence is tensor([24, 43, 58,  5, 57]), the target is 1
when input sequence is tensor([24, 43, 58,  5, 57,  1]), the target is 46
when input sequence is tensor([24, 43, 58,  5, 57,  1, 46]), the target is 43
when input sequence is tensor([24, 43, 58,  5, 57,  1, 46, 43]), the target is 39
when input sequence is tensor([44]), the target is 53
when input sequence is te

In [39]:
print(xb) # our input to the transformer

tensor([[24, 43, 58,  5, 57,  1, 46, 43],
        [44, 53, 56,  1, 58, 46, 39, 58],
        [52, 58,  1, 58, 46, 39, 58,  1],
        [25, 17, 27, 10,  0, 21,  1, 54]])


In [94]:
import torch
import torch.nn as nn
from torch.nn import functional as F

torch.manual_seed(1337)

class BigramLanguageModel(nn.Module):
    def __init__(self,vocab_size):
        super().__init__()
        self.token_embedding_table = nn.Embedding(vocab_size,vocab_size)
        
    def forward(self,idx,targets=None):
        logits = self.token_embedding_table(idx)  #B,T,C    
        if targets is None:
            loss = None
        else:
            B,T,C = logits.shape
            # 因为pytorch的cross_entropy函数要求输入(mini_batch,C)的格式，以bigram为例，C=vocab_size，实际有B*T个批次，所以需要将logits的形状变为(B*T,C)
            logits = logits.view(B*T,C)
            # targets也做相同的形状变换匹配输入格式
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits,targets)
        
        return logits,loss
    
    def generate(self,idx,max_new_tokens):
        # idx是shape为（B,T）的输入，每个T时间窗内为token的index
        for _ in range(max_new_tokens):
            logits,loss = self(idx)
            # 取出最后一个时间步的输出作为输入的下一个token
            logits = logits[:,-1,:] # becomes shape (B,C)
            prob = F.softmax(logits,dim=-1) # becomes shape (B,C)
            # 随机采样一个token
            idx_next = torch.multinomial(prob,num_samples=1) # becomes shape (B,1)
            # 增加到序列中，作为下一次输入
            idx = torch.cat((idx,idx_next),dim=-1)
        return idx

In [95]:
m = BigramLanguageModel(vocab_size=vocab_size)
logits,loss = m(xb,yb)
print(logits.shape)

torch.Size([32, 65])


In [80]:
# 可以看到的loss是4.8786，但是我们可以预估初始的loss，应该是-ln(1/65)=4.1744。
loss

tensor(4.8786, grad_fn=<NllLossBackward0>)

In [ ]:
start_idx = torch.zeros((1,1), dtype=torch.long)

torch.Size([1, 1])

In [103]:
print(decode(m.generate(idx=start_idx,max_new_tokens=100)[0].tolist()))


dRptRCPVjwW.RSVMjs-bgRkzrBTEa!!oP fRSxq.PLboTMkX'DUYepxIBFAYuxKXe.jeh
sa!3MGFrSjuM:wX!?BTMl!.?,M:bQz


In [105]:
# create a pytorch optimizer
optimizer = torch.optim.AdamW(m.parameters(),lr=1e-3)

In [109]:
batch_size = 32

for _ in range(10000):
    xb,yb = get_batch(split='train')
    
    logits,loss = m(xb,yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()
    
print(loss.item())

2.4940128326416016


In [ ]:
# 因为这些token之间还没有互相交流，所以仅仅依靠bigram模型生成的结果较差。
print(decode(m.generate(idx=start_idx,max_new_tokens=300)[0].tolist()))




Buru.
Serth'Whth:
AUESI isin. thed thmyo as ELI g.
CHUSisthethellend:
GAnd fo,
CKI:
DULird m wotiotoup-PAny, har lre uf,
HEL:
FLod bery s. angupr,

MOWe t ld g, ts m reou ano f s thatheewisto burinoura s,
Tono u t acou s, ingh cak r deftilselend wn. s acharofe sengor peey,
Cod batood bllthither; i


## The mathematic trick in self-attention

In [ ]:
torch.manual_seed(1337)
B,T,C = 4,8,2 # batch, time, channel
x = torch.randn(B,T,C)
x.shape

torch.Size([4, 8, 2])

In [179]:
# 这里的主要想法是让token之间开始产生联系communication，产生联系的数字关系是求这些token在连续时间步上的平均值
# 计算平均值是一种很弱的计算联系的方式，因为直观上来看会丢失大量的信息
# 并且当前时间步的token的与其它token的关系计算方式是有讲究的：
# 1.当前token只能与前面时间步的token产生联系，即第5个时间步的token只能与第1，2，3，4个时间步的token产生联系，因此第5个时间步的token存储的信息是前4个时间步的token加上第5个时间步自己的信息的平均值；
# 2.第5个token不能与后续的token产生联系的原因是因为我们需要根据第5个时间步的token来预测第6个时间步的token，如果第6个时间步的token或者及后续的token也参与第5个token信息平均值的计算了，那么等于是泄露答案了；
# 3.这种注意力机制的设计是为了让模型根据历史的信息来实现未来数据的预测。

xbow = torch.zeros(B, T, C) # xbow代表的是词袋模型，通常代表数据的平均值
for b in range(B):
    for t in range(T):
        xprev = x[b,:t+1] # t,C
        xbow[b,t] = torch.mean(xprev, dim=0)

In [166]:
# 这里是聚合了历史token的当前token的信息值
# 可以看到在第1个batch中，第1个时间步的token和xbow信息值都等于[ 0.1808, -0.0700]
# 而在第2个时间步的token含有的信息[-0.0894, -0.4926]是通过计算，第1个时间步的token[ 0.1808, -0.0700]和第2个时间步的token[-0.3596, -0.9152]在T维度方向的平均值。
xbow[0]

tensor([[ 0.1808, -0.0700],
        [-0.0894, -0.4926],
        [ 0.1490, -0.3199],
        [ 0.3504, -0.2238],
        [ 0.3525,  0.0545],
        [ 0.0688, -0.0396],
        [ 0.0927, -0.0682],
        [-0.0341,  0.1332]])

In [ ]:
# 这里是原始token的值
x[0]

tensor([[ 0.1808, -0.0700],
        [-0.3596, -0.9152],
        [ 0.6258,  0.0255],
        [ 0.9545,  0.0643],
        [ 0.3612,  1.1679],
        [-1.3499, -0.5102],
        [ 0.2360, -0.2398],
        [-0.9211,  1.5433]])

In [180]:
# version 2
# 应用矩阵计算提高效率：
# 设置function矩阵wei,用来计算每个时间步及其之前时间步的加权求和值
wei = torch.tril(torch.ones(T, T)) # wei = weights 意为权重参数
wei = wei/wei.sum(dim=1,keepdim=True)
print(wei.shape) # 8,8
xbow_new = wei @ x # 8,8 @ 4,8,2 -> 4,8,2
print(xbow_new.shape) # 4,8,2
print(xbow_new[0])

torch.Size([8, 8])
torch.Size([4, 8, 2])
tensor([[ 0.1808, -0.0700],
        [-0.0894, -0.4926],
        [ 0.1490, -0.3199],
        [ 0.3504, -0.2238],
        [ 0.3525,  0.0545],
        [ 0.0688, -0.0396],
        [ 0.0927, -0.0682],
        [-0.0341,  0.1332]])


In [182]:
# 检查两个tensor是否逐元素逐位置相等
torch.allclose(xbow,xbow_new, rtol=1e-4, atol=1e-4)

True

In [ ]:
# version 3
tril = torch.tril(torch.ones(T, T))
# 这里可以看成是初始各个时间步中的token之间的亲密度，初始为0，表示当前位置的token对其它时间步的token没有亲密度，
# 但是随着训练，这些token会逐渐发现对其它某些时间步的token感兴趣，因此它们的值不会一直恒定是0
wei = torch.zeros(T, T) 
# 通过下三角矩阵的mask遮蔽，保证了当前时间步的token只能从之前时间步的token中获取信息，而不能从之后的时间步获取信息
wei = wei.masked_fill(tril == 0, float('-inf'))
# 这里使用softmax将概率平均到每一个时间步的token上，使得每个时间步的token贡献权重都是相同的
wei = wei.softmax(dim=1)
xbow_3 = wei @ x # 8,8 @ 4,8,2 -> 4,8,2
torch.allclose(xbow_new,xbow_3) 

# 这里就是self-attention机制的preview,利用下三角遮蔽矩阵对过去时间步的token进行加权聚合，mask遮蔽矩阵会告诉我们，
# 当前时间步的token需要多少个历史token参与计算当前位置token的信息。

True

In [137]:
# 上述方法可以实现，但是效率不高，高效的方式是使用矩阵乘法实现
# 矩阵乘法的优势在于可以并行计算，可以大大提高运算速度。
torch.manual_seed(42)

a = torch.ones(3,3)
b = torch.randint(0,10,(3,2)).float()

c = torch.matmul(a,b)

print(f'a=\n{a}')
print(f'b=\n{b}')
print(f'c=\n{c}')

# 这里可以看到矩阵c中每一行各列的值都分别是矩阵b中的对应列的所有行的元素之和，这里可以看作矩阵a是一个function矩阵，这个function的功能就是将矩阵b中的每一列的所有行元素求和。

a=
tensor([[1., 1., 1.],
        [1., 1., 1.],
        [1., 1., 1.]])
b=
tensor([[2., 7.],
        [6., 4.],
        [6., 5.]])
c=
tensor([[14., 16.],
        [14., 16.],
        [14., 16.]])


In [ ]:
# 如果对function矩阵进行变换，那么就会应用不同的计算规则对矩阵b进行计算

a = torch.tril(a)
c = torch.matmul(a,b)

print(f'a=\n{a}')
print(f'b=\n{b}')
print(f'c=\n{c}')

# 现在可以看到c的每一行的对应列结果都是b的对应行加上比其行索引小的行的对应列的结果

a=
tensor([[1., 0., 0.],
        [1., 1., 0.],
        [1., 1., 1.]])
b=
tensor([[2., 7.],
        [6., 4.],
        [6., 5.]])
c=
tensor([[ 2.,  7.],
        [ 8., 11.],
        [14., 16.]])


In [ ]:
# 为了解决之前计算效率不足的问题，我们可以构建新的function矩阵a

a = torch.tril(torch.ones(3, 3))
sum_a = torch.sum(a,dim=1,keepdim=True)
a = a/sum_a

c = torch.matmul(a,b)

print(f'a=\n{a}')
print(f'b=\n{b}')
print(f'c=\n{c}')

# 这样，无论怎么改变b的列数，都可以按照这种规则进行满足a这种function矩阵的扩展

a=
tensor([[1.0000, 0.0000, 0.0000],
        [0.5000, 0.5000, 0.0000],
        [0.3333, 0.3333, 0.3333]])
b=
tensor([[2., 7.],
        [6., 4.],
        [6., 5.]])
c=
tensor([[2.0000, 7.0000],
        [4.0000, 5.5000],
        [4.6667, 5.3333]])
